# Practical 4B - MNIST Classification with a ANN

### Importing the libraries

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras.datasets.mnist as mnist
from keras.utils import np_utils

### Creating the Base Layer

In [3]:
# creating the base layer

class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward(self, input):
        pass

    def backward(self, output_gradient, learning_rate):
        pass

### Creating the Dense Layer

In [4]:
# creating the Dense layer

import numpy as np

class Layer_Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size,input_size)
        self.biases = np.random.randn(output_size,1)
        

    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input ) + self.biases

    def backward(self, output_gradient, learning_rate):
        weights_gradient = np.dot(output_gradient,self.input.T )
        input_gradient = np.dot(self.weights.T, output_gradient)
        self.weights -= learning_rate * weights_gradient
        self.biases -=learning_rate * output_gradient
        return input_gradient

### Creating the Activation Layer

In [5]:
# creating the activation layer

class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward(self, input):
        self.input = input
        return self.activation(self.input)

    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))

### Creating the Activation Function

In [6]:
# creating the activation functions

# Linear function
class Linear(Activation):
    def __init__(self):
        def linear(x):
            return x
        
        def linear_prime(x):
            return 1
        
        super().__init__(linear, linear_prime)

# Tanh function
class Tanh(Activation):
    def __init__(self):
        def tanh(x):
            return np.tanh(x)
        
        def tanh_prime(x):
            return 1 - np.tanh(x)**2
        
        super().__init__(tanh, tanh_prime)

# Sigmoid function
class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))
        
        def sigmoid_prime(x):
            return sigmoid(x) * (1 - sigmoid(x))
        
        super().__init__(sigmoid, sigmoid_prime)

# ReLU function
class ReLU(Activation):
    def __init__(self):
        def relu(x):
            return np.maximum(0,x)
        
        def relu_prime(x):
            return np.where(x > 0, 1, 0)
        
        super().__init__(relu, relu_prime)



### Creating the Loss Function

In [7]:
# creating the loss function

def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

# creating the loss function derivative

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)

### Creating the function to train and predict

In [8]:
def predict(network, input):
    output = input
    for layer in network:
        output = layer.forward(output)
    return output

def train(network, loss, loss_prime,x_train, y_train, epochs=10, learning_rate=0.1):
    for e in range (epochs):
        error = 0
        for x,y in zip(x_train, y_train):
            output = predict(network, x)
            error += loss(y, output)
            output_gradient = loss_prime(y, output)
            for layer in reversed(network):
                output_gradient = layer.backward(output_gradient, learning_rate)

        error  /=len(x_train)

        if verbose:
            print("Epoch: %d, Error: %.3f" % (e, error))
     

In [9]:
def preprocess_data (x,y,limit):
    x = x.reshape(x.shape[0], 28*28, 1)
    x = x.astype('float32')/255

    y = np_utils.to_categorical(y, 10)
    y  = y.reshape(y.shape[0], 10, 1)

    x_train = x[:limit]
    y_train = y[:limit]

    return x_train, y_train

In [39]:
# loading the data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 5000)
x_test, y_test = preprocess_data(x_test, y_test, 20)

### Creating the Neural Network

In [40]:
# creating the network
# dense (28*28,100)
# activation (tanh)
# dense (100,10)
# activation (sigmoid)
network = [
    Layer_Dense(28*28, 40),
    Tanh(),
    Layer_Dense(40, 10),
    Sigmoid()
]


# training the network
train(network, mse, mse_prime, x_train, y_train, epochs=100, learning_rate=0.1)   

Epoch: 0, Error: 0.129
Epoch: 1, Error: 0.088
Epoch: 2, Error: 0.079
Epoch: 3, Error: 0.073
Epoch: 4, Error: 0.069
Epoch: 5, Error: 0.066
Epoch: 6, Error: 0.063
Epoch: 7, Error: 0.061
Epoch: 8, Error: 0.058
Epoch: 9, Error: 0.055
Epoch: 10, Error: 0.053
Epoch: 11, Error: 0.051
Epoch: 12, Error: 0.050
Epoch: 13, Error: 0.048
Epoch: 14, Error: 0.047
Epoch: 15, Error: 0.046
Epoch: 16, Error: 0.045
Epoch: 17, Error: 0.045
Epoch: 18, Error: 0.044
Epoch: 19, Error: 0.043
Epoch: 20, Error: 0.042
Epoch: 21, Error: 0.042
Epoch: 22, Error: 0.041
Epoch: 23, Error: 0.040
Epoch: 24, Error: 0.040
Epoch: 25, Error: 0.039
Epoch: 26, Error: 0.039
Epoch: 27, Error: 0.038
Epoch: 28, Error: 0.038
Epoch: 29, Error: 0.037
Epoch: 30, Error: 0.037
Epoch: 31, Error: 0.037
Epoch: 32, Error: 0.036
Epoch: 33, Error: 0.036
Epoch: 34, Error: 0.036
Epoch: 35, Error: 0.036
Epoch: 36, Error: 0.035
Epoch: 37, Error: 0.035
Epoch: 38, Error: 0.035
Epoch: 39, Error: 0.034
Epoch: 40, Error: 0.034
Epoch: 41, Error: 0.034
Ep

### Testing the Neural Network

In [41]:
# testing the network
for x,y in zip(x_test, y_test):
    output = predict(network, x)
    print("Original: %d, Predicted: %d" % (np.argmax(y), np.argmax(output)))

Original: 7, Predicted: 7
Original: 2, Predicted: 2
Original: 1, Predicted: 1
Original: 0, Predicted: 0
Original: 4, Predicted: 4
Original: 1, Predicted: 1
Original: 4, Predicted: 4
Original: 9, Predicted: 5
Original: 5, Predicted: 0
Original: 9, Predicted: 7
Original: 0, Predicted: 0
Original: 6, Predicted: 6
Original: 9, Predicted: 4
Original: 0, Predicted: 0
Original: 1, Predicted: 1
Original: 5, Predicted: 3
Original: 9, Predicted: 0
Original: 7, Predicted: 7
Original: 3, Predicted: 6
Original: 4, Predicted: 4


In [42]:
# calculating the accuracy
correct = 0
for x,y in zip(x_test, y_test):
    output = predict(network, x)
    if np.argmax(y) == np.argmax(output):
        correct += 1



print("Accuracy: ",correct/len(x_test))

Accuracy:  0.65
